In [1]:
import json
import random
import argparse
import numpy as np
import torch

import src
from src.slurm import init_signal_handler, init_distributed_mode
from src.utils import bool_flag, initialize_exp
from src.model import check_model_params, build_modules
from src.envs import ENVS, build_env
from src.trainer import Trainer
from src.evaluator import Evaluator


#np.seterr(all='raise')


def get_parser():
    """
    Generate a parameters parser.
    """
    # parse parameters
    parser = argparse.ArgumentParser(description="Language transfer")

    # main parameters
    parser.add_argument("--dump_path", type=str, default="./dumped/",
                        help="Experiment dump path")
    parser.add_argument("--exp_name", type=str, default="debug",
                        help="Experiment name")
    parser.add_argument("--save_periodic", type=int, default=0,
                        help="Save the model periodically (0 to disable)")
    parser.add_argument("--exp_id", type=str, default="",
                        help="Experiment ID")

    # float16 / AMP API
    parser.add_argument("--fp16", type=bool_flag, default=False,
                        help="Run model with float16")
    parser.add_argument("--amp", type=int, default=-1,
                        help="Use AMP wrapper for float16 / distributed / gradient accumulation. Level of optimization. -1 to disable.")

    # model parameters
    parser.add_argument("--token_size", type=int, default=64,
                        help="every token_size datapoints can be seen as a token")
    parser.add_argument("--emb_dim", type=int, default=256,
                        help="Embedding layer size")
    parser.add_argument("--n_enc_layers", type=int, default=4,
                        help="Number of Transformer layers in the encoder")
    parser.add_argument("--n_dec_layers", type=int, default=4,
                        help="Number of Transformer layers in the decoder")
    parser.add_argument("--n_heads", type=int, default=4,
                        help="Number of Transformer heads")
    parser.add_argument("--dropout", type=float, default=0,
                        help="Dropout")
    parser.add_argument("--attention_dropout", type=float, default=0,
                        help="Dropout in the attention layer")
    parser.add_argument("--share_inout_emb", type=bool_flag, default=True,
                        help="Share input and output embeddings")
    parser.add_argument("--sinusoidal_embeddings", type=bool_flag, default=False,
                        help="Use sinusoidal embeddings")

    # training parameters
    parser.add_argument("--env_base_seed", type=int, default=0,
                        help="Base seed for environments (-1 to use timestamp seed)")
    parser.add_argument("--max_len", type=int, default=512,
                        help="Maximum sequences length")
    parser.add_argument("--batch_size", type=int, default=32,
                        help="Number of sentences per batch")
    parser.add_argument("--optimizer", type=str, default="adam,lr=0.0001",
                        help="Optimizer (SGD / RMSprop / Adam, etc.)")
    parser.add_argument("--clip_grad_norm", type=float, default=5,
                        help="Clip gradients norm (0 to disable)")
    parser.add_argument("--epoch_size", type=int, default=300000,
                        help="Epoch size / evaluation frequency")
    parser.add_argument("--max_epoch", type=int, default=100000,
                        help="Maximum epoch size")
    parser.add_argument("--stopping_criterion", type=str, default="",
                        help="Stopping criterion, and number of non-increase before stopping the experiment")
    parser.add_argument("--validation_metrics", type=str, default="",
                        help="Validation metrics")
    parser.add_argument("--accumulate_gradients", type=int, default=1,
                        help="Accumulate model gradients over N iterations (N times larger batch sizes)")
    parser.add_argument("--num_workers", type=int, default=10,
                        help="Number of CPU workers for DataLoader")
    parser.add_argument("--same_nb_ops_per_batch", type=bool_flag, default=False,
                        help="Generate sequences with the same number of operators in batches.")

    # export data / reload it
    parser.add_argument("--export_data", type=bool_flag, default=False,
                        help="Export data and disable training.")
    parser.add_argument("--reload_data", type=str, default="",
                        help="Load dataset from the disk (train_path1,valid_path1,test_path1)")
    parser.add_argument("--reload_size", type=int, default=-1,
                        help="Reloaded training set size (-1 for everything)")

    # environment parameters
    parser.add_argument("--env_name", type=str, default="char_sp",
                        help="Environment name")
    ENVS[parser.parse_known_args()[0].env_name].register_args(parser)

    # tasks
    #parser.add_argument("--tasks", type=str, default="",
    #                    help="Tasks")

    # beam search configuration
    parser.add_argument("--beam_eval", type=bool_flag, default=False,
                        help="Evaluate with beam search decoding.")
    parser.add_argument("--beam_size", type=int, default=1,
                        help="Beam size, default = 1 (greedy decoding)")
    parser.add_argument("--beam_length_penalty", type=float, default=1,
                        help="Length penalty, values < 1.0 favor shorter sentences, while values > 1.0 favor longer ones.")
    parser.add_argument("--beam_early_stopping", type=bool_flag, default=True,
                        help="Early stopping, stop as soon as we have `beam_size` hypotheses, although longer ones may have better scores.")

    # reload pretrained model / checkpoint
    parser.add_argument("--reload_model", type=str, default="",
                        help="Reload a pretrained model")
    parser.add_argument("--reload_checkpoint", type=str, default="",
                        help="Reload a checkpoint")

    # evaluation
    parser.add_argument("--eval_only", type=bool_flag, default=False,
                        help="Only run evaluations")
    parser.add_argument("--eval_verbose", type=int, default=0,
                        help="Export evaluation details")
    parser.add_argument("--eval_verbose_print", type=bool_flag, default=False,
                        help="Print evaluation details")

    # debug
    parser.add_argument("--debug_slurm", type=bool_flag, default=False,
                        help="Debug multi-GPU / multi-node within a SLURM job")
    parser.add_argument("--debug", help="Enable all debug flags",
                        action="store_true")

    # CPU / multi-gpu / multi-node
    parser.add_argument("--cpu", type=bool_flag, default=False,
                        help="Run on CPU")
    parser.add_argument("--local_rank", type=int, default=-1,
                        help="Multi-GPU - Local rank")
    parser.add_argument("--master_port", type=int, default=-1,
                        help="Master port (for multi-node SLURM jobs)")

    return parser



In [2]:

args=['--exp_name', 'third_train', '--reload_data',
      "data.prefix.counts.valid,data.prefix.counts.valid_100,data.prefix.counts.valid_100",
      '--reload_size' ,'238', '--token_size','64','--emb_dim' ,'128' ,'--n_enc_layers' ,'6', '--n_dec_layers' ,'6' ,'--n_heads', '8' ,
      '--optimizer', "adam,lr=0.0001" , '--batch_size', '1' ,'--epoch_size' ,'300',  '--cpu', 'true' ,'--eval_only','true',
      '--operators',"add:10,sub:3,mul:10,div:5,sqrt:4,pow2:4,pow3:2,pow4:1,pow5:1,ln:4,exp:4,sin:4,cos:4,tan:4",
     '--reload_model',"dumped/third_train/datasize_517648/periodic-1600.pth"]
args=['--exp_name', 'third_train', '--reload_data',
      "data.prefix.counts.valid,data.prefix.counts_forth.valid,data.prefix.counts_forth.valid",
      '--reload_size' ,'238', '--token_size','64','--emb_dim' ,'128' ,'--n_enc_layers' ,'6', '--n_dec_layers' ,'6' ,'--n_heads', '8' ,
      '--optimizer', "adam,lr=0.0001" , '--batch_size', '1' ,'--epoch_size' ,'300',  '--cpu', 'true' ,'--eval_only','true',
      '--operators',"add:10,sub:3,mul:10,div:5,sqrt:4,pow2:4,pow3:2,pow4:1,pow5:1,ln:4,exp:4,sin:4,cos:4,tan:4",
     '--reload_model',"dumped/forth_train/datasize_4269/periodic-1200.pth"]

parser = get_parser()
params = parser.parse_args(args)

In [3]:
def main(params):

    # initialize the multi-GPU / multi-node training
    # initialize experiment / SLURM signal handler for time limit / pre-emption
    init_distributed_mode(params)
    logger = initialize_exp(params)
    init_signal_handler()

    # CPU / CUDA
    if params.cpu:
        assert not params.multi_gpu
    else:
        assert torch.cuda.is_available()
    src.utils.CUDA = not params.cpu

    # build environment / modules / trainer / evaluator
    env = build_env(params)
    modules = build_modules(env, params)
    trainer = Trainer(modules, env, params)
    evaluator = Evaluator(trainer)

    # evaluation
    if params.eval_only:
        scores = evaluator.run_all_evals()
        for k, v in scores.items():
            logger.info("%s -> %.6f" % (k, v))
        logger.info("__log__:%s" % json.dumps(scores))
        #exit()



In [4]:


# debug mode
if params.debug:
    params.exp_name = 'debug'
    if params.exp_id == '':
        params.exp_id = 'debug_%08i' % random.randint(0, 100000000)
    params.debug_slurm = True

# check parameters
check_model_params(params)

# run experiment
main(params)

INFO - 05/24/22 13:46:50 - 0:00:00 - ============ Initialized logger ============
INFO - 05/24/22 13:46:50 - 0:00:00 - accumulate_gradients: 1
                                     amp: -1
                                     attention_dropout: 0
                                     balanced: False
                                     batch_size: 1
                                     beam_early_stopping: True
                                     beam_eval: False
                                     beam_length_penalty: 1
                                     beam_size: 1
                                     clip_grad_norm: 5
                                     command: python /root/miniconda3/lib/python3.8/site-packages/ipykernel_launcher.py '-f' '/root/.local/share/jupyter/runtime/kernel-bd325417-030f-42ad-9689-78faa33107f0.json' --exp_id "xz9bt94jqu"
                                     cpu: True
                                     datalength: 256
                                   

SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : 2c67935a531f


INFO - 05/24/22 13:46:51 - 0:00:00 - Reloading modules from dumped/forth_train/datasize_4269/periodic-1200.pth ...
INFO - 05/24/22 13:47:05 - 0:00:14 - Number of parameters (encoder): 1730304
INFO - 05/24/22 13:47:05 - 0:00:14 - Number of parameters (decoder): 2128189
INFO - 05/24/22 13:47:05 - 0:00:14 - Found 265 parameters in model.
INFO - 05/24/22 13:47:05 - 0:00:14 - Optimizers: model
INFO - 05/24/22 13:47:05 - 0:00:14 - Creating valid iterator for ...
INFO - 05/24/22 13:47:05 - 0:00:14 - Loading data from data.prefix.counts_forth.valid ...
INFO - 05/24/22 13:47:05 - 0:00:14 - Loaded 100 equations from the disk.


['data.prefix.counts.valid', 'data.prefix.counts_forth.valid', 'data.prefix.counts_forth.valid']


INFO - 05/24/22 13:47:05 - 0:00:15 - 0/100


...

y is:  ['add', 'x', 'mul', 'INT-', '1', 'pow', 'add', 'INT+', '5', 'x', 'INT+', '2', '<s>']
predict is  ['add', 'x', 'mul', 'INT-', '1', 'pow', 'add', 'INT+', '4', 'x', 'INT+', '2', '<s>']
t is:  tensor([[ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [False]])
valid is:  tensor([0])
y is:  ['add', 'INT+', '5', 'mul', 'INT-', '2', 'x', '<s>']
predict is  ['add', 'INT+', '3', 'mul', 'INT-', '2', 'x', '<s>']
t is:  tensor([[ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False]])
valid is:  tensor([0])
y is:  ['mul', 'pow', 'x', 'INT-', '1', 'add', 'INT+', '1', '5', 'mul', 'INT+', '5', 'x', '<s>']
predict is  ['mul', 'INT+', 'x', 'INT-', '1', 'add', 'INT+', '2', '6', 'mul', 'INT+', '4', 'x', '<s>']
t is:  tensor([[ True],
        [

t is:  tensor([[ True],
        [False],
        [ True],
        [False],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [False]])
valid is:  tensor([0])
y is:  ['add', 'INT+', '4', 'pow', 'add', 'INT-', '3', 'x', 'INT+', '2', '<s>']
predict is  ['add', 'INT+', '4', 'pow', 'add', 'INT+', '3', 'x', 'INT+', '2', '<s>']
t is:  tensor([[ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False]])
valid is:  tensor([0])
y is:  ['mul', 'INT+', '4', 'pow', 'add', 'div', 'INT+', '1', 'INT+', '2', 'x', 'INT-', '1', '<s>']
predict is  ['mul', 'INT+', '3', 'pow', 'add', 'div', 'INT+', '1', 'INT+', '4', 'x', 'INT-', '1', '<s>']
t is:  tensor([[ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],


y is:  ['mul', 'x', 'add', 'INT-', '4', 'mul', 'div', 'INT+', '1', 'INT+', '2', 'x', '<s>']
predict is  ['add', 'add', 'add', 'INT-', '6', 'pow', 'div', 'INT+', '1', 'INT+', '4', 'x', '<s>']
t is:  tensor([[False],
        [False],
        [ True],
        [ True],
        [False],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [False]])
valid is:  tensor([0])
y is:  ['add', 'INT-', '9', 'mul', 'div', 'INT+', '1', 'INT+', '3', 'x', '<s>']
predict is  ['add', 'INT-', '3', 'x', 'div', 'INT+', '1', 'INT+', '4', 'x', '<s>']
t is:  tensor([[ True],
        [ True],
        [False],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [False]])
valid is:  tensor([0])
y is:  ['add', 'INT+', '2', 'mul', 'INT+', '9', 'pow', 'x', 'INT+', '2', '<s>']
predict is  ['mul', 'INT+', '4', 'mul', 'INT+', '1', 'pow', 'x

y is:  ['pow', 'add', 'INT-', '3', 'pow', 'x', 'INT+', '2', 'INT+', '2', '<s>']
predict is  ['pow', 'add', 'INT+', '4', 'pow', 'x', 'INT+', '2', 'INT+', '2', '<s>']
t is:  tensor([[ True],
        [ True],
        [False],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False]])
valid is:  tensor([0])
y is:  ['add', 'INT+', '7', 'mul', 'INT+', '3', 'pow', 'x', 'INT-', '1', '<s>']
predict is  ['add', 'INT+', '5', 'mul', 'INT+', '3', 'pow', 'x', 'INT-', '1', '<s>']
t is:  tensor([[ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False]])
valid is:  tensor([0])
y is:  ['mul', 'pow', 'add', 'INT+', '5', 'mul', 'INT-', '1', 'x', 'INT-', '1', 'add', 'INT+', '3', 'x', '<s>']
predict is  ['mul', 'INT+', 'add', 'INT+', '5', 'mul', 'INT-', '1', 'x', 'INT-', '1', 'a

INFO - 05/24/22 13:47:30 - 0:00:40 - 4/100 (4.0%) equations were evaluated correctly.
INFO - 05/24/22 13:47:30 - 0:00:40 - epoch -> 0.000000
INFO - 05/24/22 13:47:30 - 0:00:40 - valid_xe_loss -> 6.963599
INFO - 05/24/22 13:47:30 - 0:00:40 - valid_acc -> 4.000000
INFO - 05/24/22 13:47:30 - 0:00:40 - valid_acc_1 -> 0.000000
INFO - 05/24/22 13:47:30 - 0:00:40 - valid_acc_2 -> 9.090909
INFO - 05/24/22 13:47:30 - 0:00:40 - valid_acc_3 -> 0.000000
INFO - 05/24/22 13:47:30 - 0:00:40 - valid_acc_4 -> 6.666667
INFO - 05/24/22 13:47:30 - 0:00:40 - valid_acc_5 -> 0.000000
INFO - 05/24/22 13:47:30 - 0:00:40 - valid_acc_6 -> 0.000000
INFO - 05/24/22 13:47:30 - 0:00:40 - test_xe_loss -> 6.963599
INFO - 05/24/22 13:47:30 - 0:00:40 - test_acc -> 4.000000
INFO - 05/24/22 13:47:30 - 0:00:40 - test_acc_1 -> 0.000000
INFO - 05/24/22 13:47:30 - 0:00:40 - test_acc_2 -> 9.090909
INFO - 05/24/22 13:47:30 - 0:00:40 - test_acc_3 -> 0.000000
INFO - 05/24/22 13:47:30 - 0:00:40 - test_acc_4 -> 6.666667
INFO - 05/2

In [7]:

parser = get_parser()
params = parser.parse_args(args)